In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, os


In [51]:
df = pd.read_csv('244400404_effectifs-eleves-cleaned')
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957


In [52]:
df['datetime']= df['Date'].apply( lambda string1 :datetime.datetime.strptime(string1,"%Y-%m-%d"))
df['joursem']= df['Date'].apply( lambda string1 :datetime.datetime.strptime(string1,"%Y-%m-%d").weekday())
df['isoweek']= df['Date'].apply( lambda string1 :datetime.datetime.strptime(string1,"%Y-%m-%d").isocalendar().week)

In [53]:
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870,2011-09-06,1,36
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698,2011-09-05,0,36
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427,2011-09-05,0,36
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169,2011-09-05,0,36
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957,2011-09-05,0,36


In [54]:
df_calend=pd.read_excel('../../data/Suppl_XLSX/conv_xlsx/fetes_juives.xlsx')
df_calend['datetime']= df_calend['Column2'].apply(lambda string1 : datetime.datetime.strptime(string1._date_repr,"%Y-%m-%d"))
df_calend['Fete Juive']=df_calend['datetime'].apply(lambda string1 : '1')
df_calend.head()

,Column1,Column2,Column3,Column4,Column5,Column6,datetime,Fete Juive
0,Dim. 20,2011-03-01,juif,Pourim,Fête des sorts. Joyeux festival de printemps é...,Les fêtes juives débutent toujours la veille à...,2011-03-01,1
1,Mar. 19,2011-04-01,juif,Pessah * (19 et 20 et jusqu'au 26 avril),"Pâque juive, libération des enfants d'Israël d...",Les fêtes juives débutent toujours la veille à...,2011-04-01,1
2,Mer. 8,2011-06-01,juif,Chavouot * (8 et 9 juin),Fête des Semaines (ou de Pentecôte) célébrant ...,Les fêtes juives débutent toujours la veille à...,2011-06-01,1
3,Mar. 9,2011-08-01,juif,Jeûne du 9 Av,Jour de jeûne au terme de trois semaines d'évo...,Les fêtes juives débutent toujours la veille à...,2011-08-01,1
4,Jeu. 29,2011-09-01,juif,Roch ha-Chanah * (29 et 30 septembre),"Nouvel An (1er Tishri) 5772, rappel de la créa...",Les fêtes juives débutent toujours la veille à...,2011-09-01,1


In [55]:
df_calend=df_calend[['datetime','Fete Juive']]

df_calend.head()

,datetime,Fete Juive
0,2011-03-01,1
1,2011-04-01,1
2,2011-06-01,1
3,2011-08-01,1
4,2011-09-01,1


In [56]:
df=pd.merge(df, df_calend, how='left', on='datetime')
df['Fete Juive'].fillna('0', inplace=True)
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek,Fete Juive
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870,2011-09-06,1,36,0
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698,2011-09-05,0,36,0
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427,2011-09-05,0,36,0
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169,2011-09-05,0,36,0
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957,2011-09-05,0,36,0


In [58]:
df[['Fete Juive','RéelNorm']].groupby('Fete Juive').mean()

,RéelNorm
Fete Juive,
0,0.992410
1,0.980575


In [60]:
df[['Fete Juive','RéelNorm']].groupby('Fete Juive').median()

,RéelNorm
Fete Juive,
0,1.0
1,1.0


In [76]:
df[df['Nom du site']=='CHATAIGNIERS'].median()

C:\Users\Eric Bertrand\AppData\Local\Temp\ipykernel_9964\1132051839.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df[df['Nom du site']=='CHATAIGNIERS'].median()
C:\Users\Eric Bertrand\AppData\Local\Temp\ipykernel_9964\1132051839.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df[df['Nom du site']=='CHATAIGNIERS'].median()


RowNo         72470.000000
Prévision        90.000000
Réel             88.000000
An             2013.000000
RéelMed         115.000000
RéelNorm          0.843478
joursem           2.000000
isoweek          22.000000
Fete Juive        0.000000
dtype: float64

In [73]:
df[df['Nom du site']=='CHATAIGNIERS'].sort_values(by='Réel')

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek,Fete Juive
118297,120609,M,2012-09-04,31.0,18.0,CHATAIGNIERS,2012,85.0,0.211765,2012-09-04,1,36,0
110554,112755,M,2012-06-21,22.0,21.0,CHATAIGNIERS,2012,85.0,0.247059,2012-06-21,3,25,0
72652,74217,M,2011-09-05,37.0,25.0,CHATAIGNIERS,2011,86.0,0.290698,2011-09-05,0,36,0
95241,97108,M,2011-05-24,54.0,41.0,CHATAIGNIERS,2011,86.0,0.476744,2011-05-24,1,21,0
3675,3707,M,2013-01-18,88.0,43.0,CHATAIGNIERS,2013,115.0,0.373913,2013-01-18,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,1908,E,2013-05-28,232.0,222.0,CHATAIGNIERS,2013,115.0,1.930435,2013-05-28,1,22,0
21283,21686,E,2013-04-11,242.0,223.0,CHATAIGNIERS,2013,115.0,1.939130,2013-04-11,3,15,0
21492,21902,E,2013-02-22,226.0,224.0,CHATAIGNIERS,2013,115.0,1.947826,2013-02-22,4,8,0
109233,111424,E,2013-05-24,239.0,227.0,CHATAIGNIERS,2013,115.0,1.973913,2013-05-24,4,21,0


In [64]:
df_calend1 = df[['Nom du site','Fete Juive','RéelNorm']].groupby(['Nom du site','Fete Juive']).median()
df_calend2= pd.pivot_table(df_calend1, index=['Nom du site'],columns='Fete Juive', values='RéelNorm', aggfunc=np.mean)
df_calend2.sort_values(by='1', ascending=True).head(30)

Fete Juive,0,1
Nom du site,,
CHATAIGNIERS,0.843478,0.791304
CHATAIGNIERS M,1.000000,0.893333
CHENE D'ARON,1.000000,0.897557
JEAN ZAY,1.000000,0.915810
GUSTAVE ROCH,1.000000,0.935980
JOSEPHINE BAKER,1.000000,0.938462
MARSAUDERIES E,1.000000,0.941909
COTE D'OR,1.000000,0.954576
ALAIN FOURNIER,1.000000,0.956358
